In [25]:
import json
import pandas as pd

# Load JSON data
with open("../data/processed/mrbench_v3_devset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Ensure data is a list
if isinstance(data, dict):
    data = [data]

# Create flattened rows for each model
rows = []
for item in data:
    for model, response_data in item["tutor_responses"].items():
        row = {
            "conversation_id": item["conversation_id"],
            "conversation_history": item["conversation_history"],
            "conversation_history_structured": item["conversation_history_structured"],
            "model": model,
            "response": response_data.get("response", None),
        }
        annotation = response_data.get("annotation", {})
        for key, value in annotation.items():
            row[f"annotation_{key}"] = value
        rows.append(row)

# Convert to DataFrame
df_flat = pd.DataFrame(rows)

df_flat = df_flat.rename(
    columns={
        "annotation_Mistake_Identification": "mistake_identification",
        "annotation_Mistake_Location": "mistake_location",
        "annotation_Providing_Guidance": "providing_guidance",
        "annotation_Actionability": "actionability",
    }
)

# Convert conversation_history_structured to string seperated by \n
def convert_to_string(row):
    if isinstance(row, list):
        return "\n".join([str(item) for item in row])
    return str(row)

df_flat["conversation"] = df_flat["conversation_history_structured"].apply(
    convert_to_string
)

# Save only the relevant columns
df_flat = df_flat[
    [
        "conversation",
        "response",
        "mistake_identification",
        "mistake_location",
        "providing_guidance",
        "actionability",
    ]
]

# Save as json
df_flat.to_json(
    "../data/processed/mrbench_v3_devset_flat.json",
    orient="records",
    force_ascii=False,
)